In [127]:
import os
import time
from riotwatcher import LolWatcher, ApiError

api_key = "GET YOUR OWN KEY"
lol_watcher = LolWatcher(api_key)
my_region = 'NA1'

## Get PUUID of player
summoner_name = "jahseh gus"
me = lol_watcher.summoner.by_name(my_region,summoner_name)
puuid = me['puuid']
NA1 = "NA1"
ARAM = 450
RANKED_SOLO = 420

## Try to get 500 ARAM match IDs of player
matches = []
for big_loop_i in range(2):
    for small_loop_i in range(5):
        start = 500*big_loop_i+100*small_loop_i
        matchlist = lol_watcher.match.matchlist_by_puuid(puuid=puuid,region=NA1,start=start,queue=RANKED_SOLO,count=100)
        matches = matches + matchlist
    time.sleep(1.2)
print(len(matches))

with open('matches.txt', 'w') as filehandle:
    for match in matches:
        filehandle.write('%s\n' % match)

320


In [128]:
import pandas as pd

games_format = pd.DataFrame(columns=['match_id','start',
                        'ally_id_1','ally_id_2','ally_id_3','ally_id_4',
                        'enemy_id_1','enemy_id_2','enemy_id_3','enemy_id_4','enemy_id_5',
                        'my_champ','my_side','my_role','support_item',
                        'ally_champ_1','ally_champ_2','ally_champ_3','ally_champ_4',
                        'ally_role_1','ally_role_2','ally_role_3','ally_role_4',
                        'enemy_champ_1','enemy_champ_2','enemy_champ_3','enemy_champ_4','enemy_champ_5',
                        'enemy_role_1','enemy_role_2','enemy_role_3','enemy_role_4','enemy_role_5',
                        'win'])

In [129]:
support_item_ids = [3862,3863,3864, ## ad damage
                    3850,3851,3852, ## ap damage
                    3858,3859,3860, ## ap tank
                    3854,3855,3856] ## ad tank

games = []
for match_id in matches:
    match = lol_watcher.match.by_id(region=NA1,match_id=match_id)
    participants = match["metadata"]["participants"]
    infodto = match["info"]["participants"]
    game_start = match["info"]["gameStartTimestamp"]
    ## Determine side of player (red or blue)
    if puuid in participants[:5]:
        blue = True
    else:
        blue = False

    ## Grabbing all of our teammates (not us!)
    ally_ids = ["","","",""]
    ally_champs = ["","","",""]
    ally_roles = ["","","",""]
    offset = 0 if blue else 5
    ii = 0
    for t in range(5):
        curr = participants[offset+t]
        if curr != puuid:
            ally_ids[ii] = curr
            ally_champs[ii] = infodto[offset+t]["championName"]
            ally_roles[ii] = infodto[offset+t]["teamPosition"]
            ii += 1
        else:
            ## write down our champ
            win = infodto[offset+t]["win"]
            my_champ = infodto[offset+t]["championName"]
            my_role = infodto[offset+t]["teamPosition"]
            support_item = False
            if my_champ == "Senna":
                for item_index in range(6):
                    slot = "item" + str(item_index)
                    item = infodto[offset+t][slot]
                    if item in support_item_ids:
                        support_item = True
                        break

    ## For legiility, initializing variables with easy names (can rewrite for performance)
    my_side = 'b' if blue else 'r'
    ally_id_1 = ally_ids[0] 
    ally_id_2 = ally_ids[1] 
    ally_id_3 = ally_ids[2] 
    ally_id_4 = ally_ids[3]
    enemy_id_1 = participants[5-offset]
    enemy_id_2 = participants[6-offset]
    enemy_id_3 = participants[7-offset]
    enemy_id_4 = participants[8-offset]
    enemy_id_5 = participants[9-offset]
    ally_champ_1 = ally_champs[0]
    ally_champ_2 = ally_champs[1]
    ally_champ_3 = ally_champs[2]
    ally_champ_4 = ally_champs[3]
    ally_role_1 = ally_roles[0]
    ally_role_2 = ally_roles[1]
    ally_role_3 = ally_roles[2]
    ally_role_4 = ally_roles[3]
    enemy_champ_1 = infodto[5-offset]["championName"]
    enemy_champ_2 = infodto[6-offset]["championName"]
    enemy_champ_3 = infodto[7-offset]["championName"]
    enemy_champ_4 = infodto[8-offset]["championName"]
    enemy_champ_5 = infodto[9-offset]["championName"]
    enemy_role_1 = infodto[5-offset]["teamPosition"]
    enemy_role_2 = infodto[6-offset]["teamPosition"]
    enemy_role_3 = infodto[7-offset]["teamPosition"]
    enemy_role_4 = infodto[8-offset]["teamPosition"]
    enemy_role_5 = infodto[9-offset]["teamPosition"]

    match_frame = pd.Series([match_id,game_start,
                            ally_id_1,ally_id_2,ally_id_3,ally_id_4,
                            enemy_id_1,enemy_id_2,enemy_id_3,enemy_id_4,enemy_id_5,
                            my_champ,my_side,my_role,support_item,
                            ally_champ_1,ally_champ_2,ally_champ_3,ally_champ_4,
                            ally_role_1,ally_role_2,ally_role_3,ally_role_4,
                            enemy_champ_1,enemy_champ_2,enemy_champ_3,enemy_champ_4,enemy_champ_5,
                            enemy_role_1,enemy_role_2,enemy_role_3,enemy_role_4,enemy_role_5,
                            win],
                            index=games_format.columns)
    # games = games.append(match_frame,ignore_index=True)
    # games = pd.concat([games, match_frame],axis=0)
    games.append(match_frame)
    time.sleep(1.2)
## Convert list to pandas DF
games = pd.DataFrame(games)

,match_id,start,ally_id_1,ally_id_2,ally_id_3,ally_id_4,enemy_id_1,enemy_id_2,enemy_id_3,enemy_id_4,...,enemy_champ_2,enemy_champ_3,enemy_champ_4,enemy_champ_5,enemy_role_1,enemy_role_2,enemy_role_3,enemy_role_4,enemy_role_5,win
0,NA1_4660474119,1684550872663,VcJsxAJrtFw5x6-dJgToajIMXfVaQ2f81DNfBx6Wnvs4ve...,12j3MNVcWY_9wRnKFs94mE772WXXecuGrq1DejMeLceE4f...,csLPuZYuf3JHemtD_6X-w4Cxxpg1_CJDsaDsHbyjkRL5Hb...,tNhy7q1YNISNgAy4uTzxicpgrSTpvPNyvzaG9foVKCaAYw...,AJjQy5n-G0BcxDIH_13TVZhoX1hkmf2PNxHVTxuJM-W42C...,YLahzvPWJkGlYcmOt2EJxPiq1f7g6FJfkhBnZu9iUaqfjD...,62YuHUbPl-xK_GQKEBwkiNy81fTn9psBlJ1yr5lKhpXqHG...,wJ-La6di0X2reoDVeQilbFrgu6f3VXbajsMXbyU3B7AQx-...,...,Gragas,Akshan,Jinx,Seraphine,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,True
1,NA1_4660434073,1684548622260,5iZEwm_Wx0hZgUG-OFPrdIq9o8BUjKy-OxIHidDTyvKQ6E...,XCXwjlfEuH09rMHiDHZ84KHEDCUAI2Pr0sJYEH11phhsdv...,AFLFPDlYHAZjXnZZxFl96ZeCP2CN244DC4-L8sOceK6Uln...,UuwzVdHVwnf-BtIEwboqOjEdJN0spTsODBYZGSakXG6gp5...,AJjQy5n-G0BcxDIH_13TVZhoX1hkmf2PNxHVTxuJM-W42C...,GB7OoiejX_vj0QWV2lc9RomgaUfFIzz6qrrjWyzBFFy59F...,u9a1TjxCZdIeBbn0XZtjvfdNycczhpub92NWsxDceuOa-p...,h8oQSkEodO1CYXZBQJIW2fw2ol0RLahyV8GHDY3I_imc0u...,...,Khazix,Azir,Jhin,Soraka,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,True
10,NA1_4657577856,1684276616941,F9cfo7dWxZYP6KPhoLSWtx5V7AKbp33EOdu3xI7Kuo2eDu...,RHjfBN6OS8SRJX63VPju8gMY3R_w03fsyoag3-CKJ039X5...,xIaG-RIvljdffnYvKlS-oOfgSHcreXlSXolkf76Epg2hpm...,iig9-tv2wNJramCfuWpSUwUo7E_1WAW1IweojLMV_x0N8Q...,JBt7-z5-95eE_xNdtKCgJgrsONJSXVpmLpcB5wgvxUJC6_...,rOGoqILHVk2ho76B1bRlLEDK03dX1X4PAxyGVRaHOtwbwL...,uBYI9L8ggK1APLaIj-rlYOUhUNBPc7dNkkAXBnBrFw_Xuz...,p-BIFK39xV6EMBQlffElpscrcrA6r9XP55-wqfJYqyop4t...,...,Hecarim,Cassiopeia,Twitch,Nautilus,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,False
14,NA1_4656788755,1684194024121,vIymg_Z4NsOMS3dNL9-RiadiV9QATyRk2S8WYqLWG4QvLz...,HkaKpkzUH3DhH7f6nCWE2Fb5fqYEYeyW0OqpY_cd-Z8eLH...,ifCXcSWGVdjH384c3cY1_cZKJY2gK_PpYKYjDFd4yPm_GW...,Zizg4pUvvhdtN512I8sBxOOwCJoe1I1ms2DNW0neWxLet9...,E2yyBEeUbd4YtUINc5X95lR1nFCsETqj8y4UA9R_n6XetB...,O9_cvXpJN9_V92OmERpipJkI0z852hIo1Cd8t8ipb3HyHs...,ZnVWHNMb5HElpos_8gXyt-db_w4gczyOkqRIvk4ul79MZQ...,kKG4Y0Nh3D5fEgYVREgMbRlaldCvZ-7FPdVK_1b5du8Abk...,...,Khazix,Syndra,Jinx,Milio,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,False
17,NA1_4653101561,1683836025914,f5lG5rribsF6orgU1u8cP52xAOgpE3U7RA-TqoIEZuN2qb...,hGbMnB5r2jPMlRjby_Q5AOzEp_kQW8qA4EPtgt9DUtGuq9...,uG0boATPx9Hfe_Wk1WClHM8WAM3JTxqM2UlmbdFZPfI3no...,p-BIFK39xV6EMBQlffElpscrcrA6r9XP55-wqfJYqyop4t...,jDcOe-bbZl_rKbzYTKjsikLts8gn3PQ_SDN_rfZJPQY8bg...,IQ-brXzBkpNqcHEUrU5InMmqjH-et1-hpBO3o1JkObZkO1...,InmrlAP6R6F5F6FYPsaDE7NadApqm0mcOEuk_ZxmkKMPJK...,-tSEpqeksACb0duiA9LaoitTPqUAZkEwemD0JFwSqBAvgk...,...,Volibear,TwistedFate,Aphelios,Rakan,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,NA1_4165972380,1641430413161,c3Z9gW8fRDYZC2DyT43QDV8-T8RvNN0Ubh2FZzpLJJKQgx...,Wj8EYzcibHdNT_biEvK24l-36SYiPHXrqF9t2s4zC6_GGI...,mkfycF374nZTeVzA-jcYiSTYA7QNt-vRvaten0IwGXsdCX...,thD5nYcwHMWplRK-zoz4wV02kpkxXWnUbE4tYbZudzzhhe...,2TZCCAfwb5S-WyUeKJWcihJpQst2vOxWvBqnaQ2SKdOtml...,jIJ8p-KbFrNanPOxdXh8A88HE5IdwKQt9dktKDZk1pZGJ-...,Of4_Sb33LgjeyXzSmv3XnBKEMh_Mc3w9YTEheDZEVg_Ca9...,-_Qc9Ilk0DnJpTdFTaYpYcnIgzHPyvnJFL2TpcvWiVsa0m...,...,Talon,Corki,Jinx,Thresh,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,False
236,NA1_4144443203,1639868227779,Bluq1bXiILwQUw5VWz26NC5rCGpt0Y3TpHS87k_4pyQYQX...,uKcHGxd4e5Pj_GgRzxlKmqI5KiEFlznyaSGIjSxqwBrnI7...,bQ567MLy7-D7Vw6rs5ptkdxoVuy75oEr_TqUDJ84BcqfUB...,NPqsFilac9IOcjb3ukJ2xbTIG6MIu5f9zDFujwVQP4AR4n...,CQOeOt4hyhF880fXwec_b8LaZ9tROWjxZFPVgLX5p3N64s...,7F4PJ_79PqETH2t1GYq_tjBEeN7NmEy7TdpYe4_yze1nC4...,q87Lh3YG9fX7HRFYI-xFFFKkgzsNOBkv4Z4uD5-nuH_urn...,CsGfwaHWWk2v_d5pdOfhZrqgJJpdZlGu3FWCpf3_jiikal...,...,Zed,Yasuo,Karthus,Soraka,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,True
237,NA1_4144377769,1639865864441,eFG_KKOmL4h1Av7Zz3dx7-ldu9paX6PUdybgvM4EqQxeLO...,7F4PJ_79PqETH2t1GYq_tjBEeN7NmEy7TdpYe4_yze1nC4...,GzkyKCMZ63DxG2kTaqA85GdpunHkZ9

In [131]:
games["account"] = "jahseh gus"
games["start"] = pd.to_datetime(games['start'],unit='ms')
games.to_csv('games_dates_roles_dq1.csv',index=False)

In [145]:
games_dk1 = pd.read_csv('games_dates_roles_dk1.csv')
games_dq1 = pd.read_csv('games_dates_roles_dq1.csv')
games = pd.concat([games_dk1,games_dq1])
games

,match_id,start,ally_id_1,ally_id_2,ally_id_3,ally_id_4,enemy_id_1,enemy_id_2,enemy_id_3,enemy_id_4,...,enemy_champ_3,enemy_champ_4,enemy_champ_5,enemy_role_1,enemy_role_2,enemy_role_3,enemy_role_4,enemy_role_5,win,account
0,NA1_4661400560,2023-05-21 02:05:04.515,JI2nhWcxM9l42vFsRM3dUH3YYjxbu7unCg5Kjf8gKvmTMl...,fHDUdXXWG_qA-o-OgCH9NARUYyaPIW_jQ82c01_CyyV_g8...,wGBSjGPXHUaMhhMUvu9XqhsGGWbPXiWpuVaXPt39mvIKYu...,jIIVdKK1KZ9sW-kbSyDIhpIs_pkUaVRYl_wh3JZHSqzq5-...,9ZZOxxNWUasPytrmRBje0N0BuvlaSKnirc9hFu4eNpG-B-...,GB7OoiejX_vj0QWV2lc9RomgaUfFIzz6qrrjWyzBFFy59F...,h3oNZNhZ6_5ovUge748rS6PKGgKej0CY-JXTJqhhHHXZQO...,8ArQ3-r_kYL46f_m9sYHgy_rSn-Oa6fPldgT7BfpSMHgGy...,...,Lissandra,Varus,Thresh,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,True,duoking1
1,NA1_4661360306,2023-05-21 01:21:56.164,G7GyRdw80KvXVnr_Bf5RYU0ZyjAiLi6gCoTYQ7l4EKSiws...,wGBSjGPXHUaMhhMUvu9XqhsGGWbPXiWpuVaXPt39mvIKYu...,h3oNZNhZ6_5ovUge748rS6PKGgKej0CY-JXTJqhhHHXZQO...,hZBGO4IpGSZpKsqA2p5y_GccXMsOH9nLL0vkqAvEaGR54n...,TLgQTsUE_ZVGUMrvh-gH7Mi2Jb5rFYn1w0VgmwVpYMPqL7...,_02gZ6cSYfEXtVr-8MH1NPYcS3syLt-w2lREQoplWwzken...,JI2nhWcxM9l42vFsRM3dUH3YYjxbu7unCg5Kjf8gKvmTMl...,hDO4PxNLpBcYC9e8TZDNdLyhQREqxz32BEx7zBlTDUWrqc...,...,Renekton,Jhin,Karma,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,True,duoking1
2,NA1_4661306593,2023-05-21 00:27:25.875,yyNwa3KkvESfXwdnqeOv5ZnIJh8WqlLWe5pkRjy4CInZK8...,NrwMFPIyJmj77iHov9xYM3DFgD9XwifLmYUT_6ETrn9HRz...,F6l5cYQrKW4d3tRU39xFix2EYwnNSGlMcD0I6v9fodlgop...,sFx_drVi29XqrEKB2LAx6UGvmiBoorDDx7vEDFlOxT9mm9...,4kBrpukVVt66PCrAqaJvtmWr1pzejb_JQle_MnQr0J58xD...,QzQd503tOEC4CMjqBx2Nbk11wjHavI33NI351bpDUd2ABV...,hDO4PxNLpBcYC9e8TZDNdLyhQREqxz32BEx7zBlTDUWrqc...,9pgovaD3JwFyxeLCvuArfDgSTo81YTz-W-lvku9TLiF917...,...,Sylas,Kaisa,Milio,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,False,duoking1
3,NA1_4661286369,2023-05-21 00:03:33.915,4kBrpukVVt66PCrAqaJvtmWr1pzejb_JQle_MnQr0J58xD...,VPS6aTCAHA6IeUYtWMQKg8nsaCNTIvzTrNHGNoh9DD81DX...,HkaKpkzUH3DhH7f6nCWE2Fb5fqYEYeyW0OqpY_cd-Z8eLH...,BqObVX97_pzkE82KdHgrwKqbLNGOHAgFSdX2RzU4wk4yv1...,Yv5gxC2RlkDvjk-_cbzwqJGYvqdLv8pmqIJxvUowzCgkr4...,fHDUdXXWG_qA-o-OgCH9NARUYyaPIW_jQ82c01_CyyV_g8...,u0IkfCjrZ1RT83r6ccGmgtPW9GLyPzMhAB0KgnzI39V-hJ...,ocF0PJWJ9C6DNWraueJFhyhix7201uZFAlixC3iFL-X21O...,...,Ahri,Jhin,Nautilus,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,True,duoking1
4,NA1_4661243940,2023-05-20 23:07:05.936,4kBrpukVVt66PCrAqaJvtmWr1pzejb_JQle_MnQr0J58xD...,UCXxCP2ua-dPopu9h3ez01hclCKj1uRUcsb2U5fQXjY6gy...,uBYI9L8ggK1APLaIj-rlYOUhUNBPc7dNkkAXBnBrFw_Xuz...,v9n7P4UupQ2AHWho4ey8yNg7JLTg3kbJYmRM5I3xqRGnHb...,t1x0LSWtJ-DbCk1w87nPtcgOZ0-YvdQ1P6JJjWBKRPUDyP...,JNwe09JoTXd4Z_IXfv5hiice2ALXQvOhiomGKLPrPvUEZH...,Jn4x22UaPWsJOIS8eCU-Ab3mGohrDWtBFd8Xz45p5YRBy5...,-OB3aNmoRedM9pu6yatd2EftQp1gQVMN6NDs_0aVN1E3hH...,...,Renekton,Ezreal,Rakan,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,False,duoking1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,NA1_4058283474,2021-10-01 04:55:40.298,hMH_MR2neqgbX-N0P9-VKFso6bgSC7lr5Wa0oIv5nYmABY...,Tj5a4yDNza03o7nHVaktU31lQMtCncLqvcV7RsQLkj23kp...,Ntdd08u9cXa8NXJP3rRXftHPrBkrWmDz7twQbxT4Ti9o9D...,K3oQcvclgFPaqQs6YvJR7LE0zng5HUNxzCpVjExsrtGdZv...,_B2ZbCcRS_JHz9DYHz1Pf37IKKk-_DK1tnrRvc12r6_ELL...,b6P2aOXXVp0YoICIImC-1QUGl3wH1RugzOik8JP3-BXoei...,uv6L9YiK43Q-Hnpi-dJ9BGozOo3j_6T-oKcbXPbT3r1Xxd...,Aoq4myaFHMnl-yfRYi2y_qjUGQl9x1ob40e0xXZ3qLfyqF...,...,Azir,Caitlyn,Lulu,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,True,jahseh gus
316,NA1_4058208705,2021-10-01 04:18:18.059,rGIC3qj4oEwGYj3zstIy12NLvOjdelixVo3U_kdgQ4NZcx...,Tj5a4yDNza03o7nHVaktU31lQMtCncLqvcV7RsQLkj23kp...,R-mOeFfeM1G6Z_CioO1RLFJ6LZD-pfPoovAxeulEoI-Q7g...,bllnTeoDlLiYj3xSDcML7VaSxYav12Gy5Um8r4I5NbAShG...,U0EYGmMvspDiCwO92QB-XeQT3Jt1q8tEvku9hsP_ZlUkvD...,mjYyQXXWfHRltP0lsm38d__FA3tqybyY5q9_FZ0iRWFKOz...,_Qon0konVMaSDAhNZONtb4v78c32RTqlvEYCVYDQ553BkG...,GkkwKbrzo4GAPGCgdD_4rrFawLbgso9yLC95QSEQ9OsTF9...,...,Kassadin,MissFortune,Elise,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,False,jahseh gus
317,NA1_4058240882,2021-10-01 03:37:08.035,RVBWae2gruWjFjQJDnYOnEdHLiDBvx3ziqTO7h8QnjKC

In [146]:
games.loc[games.support_item == True, 'my_role'] = "UTILITY"
games.loc[(games.support_item == True) & (games.ally_role_1 == "UTILITY"), "ally_role_1"] = "BOTTOM"
games.loc[(games.support_item == True) & (games.ally_role_2 == "UTILITY"), "ally_role_2"] = "BOTTOM"
games.loc[(games.support_item == True) & (games.ally_role_3 == "UTILITY"), "ally_role_3"] = "BOTTOM"
games.loc[(games.support_item == True) & (games.ally_role_4 == "UTILITY"), "ally_role_4"] = "BOTTOM"

In [147]:
games

,match_id,start,ally_id_1,ally_id_2,ally_id_3,ally_id_4,enemy_id_1,enemy_id_2,enemy_id_3,enemy_id_4,...,enemy_champ_3,enemy_champ_4,enemy_champ_5,enemy_role_1,enemy_role_2,enemy_role_3,enemy_role_4,enemy_role_5,win,account
0,NA1_4661400560,2023-05-21 02:05:04.515,JI2nhWcxM9l42vFsRM3dUH3YYjxbu7unCg5Kjf8gKvmTMl...,fHDUdXXWG_qA-o-OgCH9NARUYyaPIW_jQ82c01_CyyV_g8...,wGBSjGPXHUaMhhMUvu9XqhsGGWbPXiWpuVaXPt39mvIKYu...,jIIVdKK1KZ9sW-kbSyDIhpIs_pkUaVRYl_wh3JZHSqzq5-...,9ZZOxxNWUasPytrmRBje0N0BuvlaSKnirc9hFu4eNpG-B-...,GB7OoiejX_vj0QWV2lc9RomgaUfFIzz6qrrjWyzBFFy59F...,h3oNZNhZ6_5ovUge748rS6PKGgKej0CY-JXTJqhhHHXZQO...,8ArQ3-r_kYL46f_m9sYHgy_rSn-Oa6fPldgT7BfpSMHgGy...,...,Lissandra,Varus,Thresh,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,True,duoking1
1,NA1_4661360306,2023-05-21 01:21:56.164,G7GyRdw80KvXVnr_Bf5RYU0ZyjAiLi6gCoTYQ7l4EKSiws...,wGBSjGPXHUaMhhMUvu9XqhsGGWbPXiWpuVaXPt39mvIKYu...,h3oNZNhZ6_5ovUge748rS6PKGgKej0CY-JXTJqhhHHXZQO...,hZBGO4IpGSZpKsqA2p5y_GccXMsOH9nLL0vkqAvEaGR54n...,TLgQTsUE_ZVGUMrvh-gH7Mi2Jb5rFYn1w0VgmwVpYMPqL7...,_02gZ6cSYfEXtVr-8MH1NPYcS3syLt-w2lREQoplWwzken...,JI2nhWcxM9l42vFsRM3dUH3YYjxbu7unCg5Kjf8gKvmTMl...,hDO4PxNLpBcYC9e8TZDNdLyhQREqxz32BEx7zBlTDUWrqc...,...,Renekton,Jhin,Karma,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,True,duoking1
2,NA1_4661306593,2023-05-21 00:27:25.875,yyNwa3KkvESfXwdnqeOv5ZnIJh8WqlLWe5pkRjy4CInZK8...,NrwMFPIyJmj77iHov9xYM3DFgD9XwifLmYUT_6ETrn9HRz...,F6l5cYQrKW4d3tRU39xFix2EYwnNSGlMcD0I6v9fodlgop...,sFx_drVi29XqrEKB2LAx6UGvmiBoorDDx7vEDFlOxT9mm9...,4kBrpukVVt66PCrAqaJvtmWr1pzejb_JQle_MnQr0J58xD...,QzQd503tOEC4CMjqBx2Nbk11wjHavI33NI351bpDUd2ABV...,hDO4PxNLpBcYC9e8TZDNdLyhQREqxz32BEx7zBlTDUWrqc...,9pgovaD3JwFyxeLCvuArfDgSTo81YTz-W-lvku9TLiF917...,...,Sylas,Kaisa,Milio,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,False,duoking1
3,NA1_4661286369,2023-05-21 00:03:33.915,4kBrpukVVt66PCrAqaJvtmWr1pzejb_JQle_MnQr0J58xD...,VPS6aTCAHA6IeUYtWMQKg8nsaCNTIvzTrNHGNoh9DD81DX...,HkaKpkzUH3DhH7f6nCWE2Fb5fqYEYeyW0OqpY_cd-Z8eLH...,BqObVX97_pzkE82KdHgrwKqbLNGOHAgFSdX2RzU4wk4yv1...,Yv5gxC2RlkDvjk-_cbzwqJGYvqdLv8pmqIJxvUowzCgkr4...,fHDUdXXWG_qA-o-OgCH9NARUYyaPIW_jQ82c01_CyyV_g8...,u0IkfCjrZ1RT83r6ccGmgtPW9GLyPzMhAB0KgnzI39V-hJ...,ocF0PJWJ9C6DNWraueJFhyhix7201uZFAlixC3iFL-X21O...,...,Ahri,Jhin,Nautilus,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,True,duoking1
4,NA1_4661243940,2023-05-20 23:07:05.936,4kBrpukVVt66PCrAqaJvtmWr1pzejb_JQle_MnQr0J58xD...,UCXxCP2ua-dPopu9h3ez01hclCKj1uRUcsb2U5fQXjY6gy...,uBYI9L8ggK1APLaIj-rlYOUhUNBPc7dNkkAXBnBrFw_Xuz...,v9n7P4UupQ2AHWho4ey8yNg7JLTg3kbJYmRM5I3xqRGnHb...,t1x0LSWtJ-DbCk1w87nPtcgOZ0-YvdQ1P6JJjWBKRPUDyP...,JNwe09JoTXd4Z_IXfv5hiice2ALXQvOhiomGKLPrPvUEZH...,Jn4x22UaPWsJOIS8eCU-Ab3mGohrDWtBFd8Xz45p5YRBy5...,-OB3aNmoRedM9pu6yatd2EftQp1gQVMN6NDs_0aVN1E3hH...,...,Renekton,Ezreal,Rakan,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,False,duoking1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,NA1_4058283474,2021-10-01 04:55:40.298,hMH_MR2neqgbX-N0P9-VKFso6bgSC7lr5Wa0oIv5nYmABY...,Tj5a4yDNza03o7nHVaktU31lQMtCncLqvcV7RsQLkj23kp...,Ntdd08u9cXa8NXJP3rRXftHPrBkrWmDz7twQbxT4Ti9o9D...,K3oQcvclgFPaqQs6YvJR7LE0zng5HUNxzCpVjExsrtGdZv...,_B2ZbCcRS_JHz9DYHz1Pf37IKKk-_DK1tnrRvc12r6_ELL...,b6P2aOXXVp0YoICIImC-1QUGl3wH1RugzOik8JP3-BXoei...,uv6L9YiK43Q-Hnpi-dJ9BGozOo3j_6T-oKcbXPbT3r1Xxd...,Aoq4myaFHMnl-yfRYi2y_qjUGQl9x1ob40e0xXZ3qLfyqF...,...,Azir,Caitlyn,Lulu,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,True,jahseh gus
316,NA1_4058208705,2021-10-01 04:18:18.059,rGIC3qj4oEwGYj3zstIy12NLvOjdelixVo3U_kdgQ4NZcx...,Tj5a4yDNza03o7nHVaktU31lQMtCncLqvcV7RsQLkj23kp...,R-mOeFfeM1G6Z_CioO1RLFJ6LZD-pfPoovAxeulEoI-Q7g...,bllnTeoDlLiYj3xSDcML7VaSxYav12Gy5Um8r4I5NbAShG...,U0EYGmMvspDiCwO92QB-XeQT3Jt1q8tEvku9hsP_ZlUkvD...,mjYyQXXWfHRltP0lsm38d__FA3tqybyY5q9_FZ0iRWFKOz...,_Qon0konVMaSDAhNZONtb4v78c32RTqlvEYCVYDQ553BkG...,GkkwKbrzo4GAPGCgdD_4rrFawLbgso9yLC95QSEQ9OsTF9...,...,Kassadin,MissFortune,Elise,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,False,jahseh gus
317,NA1_4058240882,2021-10-01 03:37:08.035,RVBWae2gruWjFjQJDnYOnEdHLiDBvx3ziqTO7h8QnjKC

In [104]:
games[(games.my_champ == "Caitlyn") & (games.start >= 1672560000) & (games.account == "duoking1")]["my_csm"]

11     0.263014
14     0.838471
16     0.661613
25     0.128411
26     0.381558
         ...   
553    0.462428
554    0.478326
556    0.000000
557    0.349650
558    0.000000
Name: my_csm, Length: 171, dtype: float64

In [148]:
games.to_csv('games_dates_roles_both.csv')

In [149]:
games_ally = games[['account','start',
                    'my_champ','my_role',
                    'ally_champ_1','ally_champ_2','ally_champ_3','ally_champ_4',
                    'ally_role_1','ally_role_2','ally_role_3','ally_role_4',
                    'win']].copy()
games_ally["win"] = games_ally["win"].replace({True: 1, False: 0})
games_ally["count"] = 1
games_ally["ally"] = True
games_ally_new = []
for ii in range(4):
    ally_champ_i = 'ally_champ_' + str(ii+1)
    ally_role_i = 'ally_role_' + str(ii+1)
    new = games_ally[['account','start','my_champ','my_role','ally',ally_champ_i,ally_role_i,'win','count']]
    new.columns = ['account','start','my_champ','my_role','ally','champ','role','win','count']
    games_ally_new.append(new)
games_ally_new = pd.concat(games_ally_new,ignore_index=True)

In [150]:
games_ally_new

,account,start,my_champ,my_role,ally,champ,role,win,count
0,duoking1,2023-05-21 02:05:04.515,Zyra,UTILITY,True,Akali,TOP,1,1
1,duoking1,2023-05-21 01:21:56.164,Zyra,UTILITY,True,Camille,TOP,1,1
2,duoking1,2023-05-21 00:27:25.875,Nautilus,UTILITY,True,Gragas,TOP,0,1
3,duoking1,2023-05-21 00:03:33.915,Braum,UTILITY,True,Camille,TOP,1,1
4,duoking1,2023-05-20 23:07:05.936,Nautilus,UTILITY,True,Tristana,TOP,0,1
...,...,...,...,...,...,...,...,...,...
3515,jahseh gus,2021-10-01 04:55:40.298,Tristana,BOTTOM,True,Nami,UTILITY,1,1
3516,jahseh gus,2021-10-01 04:18:18.059,Blitzcrank,UTILITY,True,Jhin,BOTTOM,0,1
3517,jahseh gus,2021-10-01 03:37:08.035,Sona,UTILITY,True,MissFortune,BOTTOM,1,1
3518,jahseh gus,2021-10-01 02:56:37.393,Sona,UTILITY,True,Varus,BOTTOM,1,1


In [151]:
games_enemy = games[['account','start',
                    'my_champ','my_role',
                    'enemy_champ_1','enemy_champ_2','enemy_champ_3','enemy_champ_4','enemy_champ_5',
                    'enemy_role_1','enemy_role_2','enemy_role_3','enemy_role_4','enemy_role_5',
                    'win']].copy()
games_enemy["win"] = games_enemy["win"].replace({True: 0, False: 1})
games_enemy["count"] = 1
games_enemy["ally"] = False
games_enemy_new = []
for ii in range(5):
    enemy_champ_i = 'enemy_champ_' + str(ii+1)
    enemy_role_i = 'enemy_role_' + str(ii+1)
    new = games_enemy[['account','start','my_champ','my_role','ally',enemy_champ_i,enemy_role_i,'win','count']]
    new.columns = ['account','start','my_champ','my_role','ally','champ','role','win','count']
    games_enemy_new.append(new)
games_enemy_new = pd.concat(games_enemy_new,ignore_index=True)

In [152]:
games_enemy_new

,account,start,my_champ,my_role,ally,champ,role,win,count
0,duoking1,2023-05-21 02:05:04.515,Zyra,UTILITY,False,Gnar,TOP,0,1
1,duoking1,2023-05-21 01:21:56.164,Zyra,UTILITY,False,Mordekaiser,TOP,0,1
2,duoking1,2023-05-21 00:27:25.875,Nautilus,UTILITY,False,KSante,TOP,1,1
3,duoking1,2023-05-21 00:03:33.915,Braum,UTILITY,False,Ornn,TOP,0,1
4,duoking1,2023-05-20 23:07:05.936,Nautilus,UTILITY,False,Jax,TOP,1,1
...,...,...,...,...,...,...,...,...,...
4395,jahseh gus,2021-10-01 04:55:40.298,Tristana,BOTTOM,False,Lulu,UTILITY,0,1
4396,jahseh gus,2021-10-01 04:18:18.059,Blitzcrank,UTILITY,False,Elise,UTILITY,1,1
4397,jahseh gus,2021-10-01 03:37:08.035,Sona,UTILITY,False,Xerath,UTILITY,0,1
4398,jahseh gus,2021-10-01 02:56:37.393,Sona,UTILITY,False,Nami,UTILITY,0,1


In [153]:
full_games = pd.concat([games_ally_new,games_enemy_new])

In [155]:
full_games_s13 = full_games[full_games.start >= '2023-01-11']
full_games_s13

,account,start,my_champ,my_role,ally,champ,role,win,count
0,duoking1,2023-05-21 02:05:04.515,Zyra,UTILITY,True,Akali,TOP,1,1
1,duoking1,2023-05-21 01:21:56.164,Zyra,UTILITY,True,Camille,TOP,1,1
2,duoking1,2023-05-21 00:27:25.875,Nautilus,UTILITY,True,Gragas,TOP,0,1
3,duoking1,2023-05-21 00:03:33.915,Braum,UTILITY,True,Camille,TOP,1,1
4,duoking1,2023-05-20 23:07:05.936,Nautilus,UTILITY,True,Tristana,TOP,0,1
...,...,...,...,...,...,...,...,...,...
4121,jahseh gus,2023-03-25 01:49:43.022,Senna,UTILITY,False,Bard,UTILITY,0,1
4122,jahseh gus,2023-03-25 01:24:49.126,Caitlyn,BOTTOM,False,Pyke,UTILITY,1,1
4123,jahseh gus,2023-02-01 07:39:38.984,Zyra,BOTTOM,False,Bard,UTILITY,1,1
4124,jahseh gus,2023-01-19 19:30:21.924,Syndra,MIDDLE,False,Lulu,UTILITY,0,1


In [156]:
full_games_s13.to_csv('champ_role_data_both_s13.csv')

In [61]:
games_enemy = games[['enemy_champ_1','enemy_champ_2','enemy_champ_3','enemy_champ_4','enemy_champ_5','win']].copy()
games_enemy["win"] = games_enemy["win"].replace({True: 0, False: 1})
games_enemy["count"] = 1
games_ally["ally"] = False
games_enemy_new = []
for ii in range(5):
    enemy_champ_i = 'enemy_champ_' + str(ii+1)
    new = games_enemy[[enemy_champ_i,'win','count']]
    new.columns = ['champ','enemy_win','enemy_count']
    games_enemy_new.append(new)
games_enemy_new = pd.concat(games_enemy_new,ignore_index=True)
enemy_sums = games_enemy_new.groupby(['champ']).sum()

In [63]:
champ_counts = pd.merge(ally_sums,enemy_sums,on='champ',how='outer').fillna(0)
champ_counts

,ally_win,ally_count,enemy_win,enemy_count
champ,,,,
Aatrox,3.0,3.0,3.0,6.0
Ahri,6.0,8.0,3.0,8.0
Akali,1.0,4.0,3.0,6.0
Akshan,5.0,9.0,5.0,7.0
Alistar,2.0,2.0,7.0,10.0
...,...,...,...,...
RekSai,0.0,0.0,1.0,4.0
Skarner,0.0,0.0,0.0,1.0
Sona,0.0,0.0,1.0,3.0


In [64]:
champ_counts["total_win"] = champ_counts["ally_win"] + champ_counts["enemy_win"]
champ_counts["total_count"] = champ_counts["ally_count"] + champ_counts["enemy_count"]
champ_counts["ally_win"] = champ_counts["ally_win"] / champ_counts["ally_count"]
champ_counts["enemy_win"] = champ_counts["enemy_win"] / champ_counts["enemy_count"]
champ_counts["total_win"] = champ_counts["total_win"] / champ_counts["total_count"]
champ_counts = champ_counts.fillna(0).sort_values(by='champ')
champ_counts.to_csv("champ_counts.csv")

In [57]:
champ_counts

,ally_win,ally_count,enemy_win,enemy_count,total_win,total_count
champ,,,,,,
Aatrox,0.111111,3.0,0.013889,6.0,0.046296,9.0
Ahri,0.011719,8.0,0.005859,8.0,0.008789,16.0
Akali,0.015625,4.0,0.013889,6.0,0.014583,10.0
Akshan,0.006859,9.0,0.014577,7.0,0.010236,16.0
Alistar,0.250000,2.0,0.007000,10.0,0.047500,12.0
...,...,...,...,...,...,...
Zeri,0.003005,11.0,0.002915,7.0,0.002970,18.0
Ziggs,0.250000,2.0,0.000000,5.0,0.071429,7.0
Zilean,1.000000,1.0,0.250000,2.0,0.500000,3.0


In [46]:
id_sums_relevant = id_sums[id_sums.total_count >= 5].sort_values('total_count',ascending=False)
id_sums_relevant

,ally_win,ally_count,enemy_win,enemy_count,total_win,total_count
id,,,,,,
UCXxCP2ua-dPopu9h3ez01hclCKj1uRUcsb2U5fQXjY6gyWKIc11ulF4niI-EwNQTA7O7BQqyrGudA,6,10,4,6,8,16
amzCw79znENELIUiVdGfqJTDe6jlJhFFxA03yjKchJ5yM4uGpMzyjokPgS_2JPLIAhZqADudzRWipA,2,5,2,6,6,11
h3oNZNhZ6_5ovUge748rS6PKGgKej0CY-JXTJqhhHHXZQOGTgAGKOJkOUTHHaNFnhnwgm_zY7V6VHg,5,6,3,5,7,11
AKWVwS5AKoN3GlPEi8M9GSxr4D1qVeY_dwDM5hIp-xkm0gG1B9x4tC6UHkv5Q2402hqrEL0sfbyfXg,3,4,3,7,7,11
OuMgG87brAjyf1VfYmHXHYOIpIK-ocVCZ5wThvhQ1MyQBgWZO-K0mT6VM9LGHAYWyh6r8XDLPXWjtQ,1,4,4,6,3,10
...,...,...,...,...,...,...
X4ZWIgScPfCEkhciBH4lSG1AJgWGWQasEA14CPz2coqbHLYc0uEoSmDGoouhSeiUu0ImQ6mILirTGQ,1,2,1,3,3,5
YZ-HFZdnHSCemQdQs2y4JLGzvWLoxwNTOv8VXRl4qpbi-_aY9WlHO9HFU4iJT3G0_HYou7uCEzvyBA,1,2,1,3,3,5
bHrDikb01hYFfTwJQz0R4OSFXaPDxW2x2QJGduMMn9Qq2Bv1HDupJ2OpTBE5f0CcQEyP0dhbE3jKHg,2,2,1,3,4,5


In [47]:
import os
import time
from riotwatcher import LolWatcher, ApiError

api_key = "GET YOUR OWN KEY"
lol_watcher = LolWatcher(api_key)
my_region = 'NA1'
summoners = []
for puuid in id_sums_relevant.index:
    dude = lol_watcher.summoner.by_puuid(encrypted_puuid=puuid,region="NA1")
    summoner_name = dude["name"]
    summoners.append(summoner_name)
    time.sleep(1.2)

In [48]:
id_sums_relevant["Summoner Name"] = summoners
id_sums_relevant["total_win"] = id_sums_relevant["ally_win"] + id_sums_relevant["enemy_win"]
id_sums_relevant["total_count"] = id_sums_relevant["ally_count"] + id_sums_relevant["enemy_count"]
id_sums_relevant["ally_win"] = id_sums_relevant["ally_win"] / id_sums_relevant["ally_count"]
id_sums_relevant["enemy_win"] = id_sums_relevant["enemy_win"] / id_sums_relevant["enemy_count"]
id_sums_relevant["total_win"] = id_sums_relevant["total_win"] / id_sums_relevant["total_count"]
id_sums_relevant.to_csv("id_sums_relevant.csv")

                                                    ally_win  ally_count   
id                                                                         
UCXxCP2ua-dPopu9h3ez01hclCKj1uRUcsb2U5fQXjY6gyW...  0.600000          10  \
amzCw79znENELIUiVdGfqJTDe6jlJhFFxA03yjKchJ5yM4u...  0.400000           5   
h3oNZNhZ6_5ovUge748rS6PKGgKej0CY-JXTJqhhHHXZQOG...  0.833333           6   
AKWVwS5AKoN3GlPEi8M9GSxr4D1qVeY_dwDM5hIp-xkm0gG...  0.750000           4   
OuMgG87brAjyf1VfYmHXHYOIpIK-ocVCZ5wThvhQ1MyQBgW...  0.250000           4   
...                                                      ...         ...   
X4ZWIgScPfCEkhciBH4lSG1AJgWGWQasEA14CPz2coqbHLY...  0.500000           2   
YZ-HFZdnHSCemQdQs2y4JLGzvWLoxwNTOv8VXRl4qpbi-_a...  0.500000           2   
bHrDikb01hYFfTwJQz0R4OSFXaPDxW2x2QJGduMMn9Qq2Bv...  1.000000           2   
dcrcf9eNpNL9pQPDycXOkgdkbHN8TgaYjcSwhEJoQBbu0tc...  1.000000           2   
zoP2VHuBwAs7jDp7NFnQjq761qssqJBlvAPOIZ-gTto1TqZ...  0.333333           3   

           

In [24]:
import os
import time
from riotwatcher import LolWatcher, ApiError

api_key = "RGAPI-a50ef4d9-fbd5-43e3-98db-4f27906915e7"
lol_watcher = LolWatcher(api_key)
my_region = 'NA1'


In [60]:
enemy_ids_sums = ids_enemy_new.groupby(['id']).sum().sort_values('enemy_count',ascending=False)

In [63]:
enemy_ids_sums[enemy_ids_sums.enemy_count >= 5]

,enemy_win,enemy_count
id,,
wap1DpTaJDZ2PnRQRMQBwJcp5-fjGq3HFc-0m6sB6qz55w2Zcq9Gba_zXYDmPRJbCwn3kSc469LziA,3,7
onZP0tygdtfiop8b4WvFubrBGpeBaxV54eKJ6HJmGHE-zVoMBFoVeGDporWiXKDlBmbOe4xE9VL9AQ,5,7
OuMgG87brAjyf1VfYmHXHYOIpIK-ocVCZ5wThvhQ1MyQBgWZO-K0mT6VM9LGHAYWyh6r8XDLPXWjtQ,2,6
MBaAn00hSXfmU9wsN_zGI0yNRy791BKc80U-b5c04_438j4J3q5pBOJKOZcqbqM7DnHv1H9UnHdmew,4,6
eVXs22zSxLFOaClYDfA8C2Cn640jbbv1wqdf9KcfYwYJ1Vl2qJcY_QTmGujz8YOEB4lvSFSGNeAgVA,5,6
amzCw79znENELIUiVdGfqJTDe6jlJhFFxA03yjKchJ5yM4uGpMzyjokPgS_2JPLIAhZqADudzRWipA,4,6
4xSQF-tFIYK_YXM9wkR1QRwmhNO-bcdByCfHIkXSN6aBBMMiWDJtUoOf7_wPd4Q_m41Ukw5Bbbz7WA,3,6
AKWVwS5AKoN3GlPEi8M9GSxr4D1qVeY_dwDM5hIp-xkm0gG1B9x4tC6UHkv5Q2402hqrEL0sfbyfXg,4,6
pSzPod5MXiqzw8L-arE9hdEFeeDkYvOnIFXklHVAxznmEuLfaPpTkujZf_GD22f6NewmEOCf9drkPw,0,5
